In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from skimage import io, transform
from sklearn.metrics import confusion_matrix
import seaborn as sns
import torchvision.models as models

In [5]:
%%time


def load_data(folder_path):
    images = []
    labels = []
    dominant_colors = []

    for color_folder in os.listdir(folder_path):
        color_path = os.path.join(folder_path, color_folder)
        if os.path.isdir(color_path):
            print(f"\nProcessing images in {color_folder} folder:")


            for image_file in os.listdir(color_path):
                image_path = os.path.join(color_path, image_file)
                if image_file.lower() == '.ds_store':
                    print(f"Skipping {image_path}")
                    continue

                try:
                    valid_extensions = ['.jpg', '.jpeg', '.png']
                    if os.path.isfile(image_path) and any(image_path.lower().endswith(ext) for ext in valid_extensions):
                        image = io.imread(image_path)
                        images.append(transform.resize(image, (100, 100)))  
                        labels.append(color_folder)

                      

                except Exception as e:
                    print(f"Error loading image {image_path}: {e}")

    return images, labels


train_folder_path = '/Users/tolga/Desktop/CS464/archive/train'
validation_folder_path = '/Users/tolga/Desktop/CS464/archive/val'

train_images, train_labels = load_data(train_folder_path)
validation_images, validation_labels = load_data(validation_folder_path)


Processing images in brown folder:

Processing images in green folder:

Processing images in blue folder:

Processing images in silver folder:

Processing images in grey folder:

Processing images in pink folder:

Processing images in red folder:

Processing images in gold folder:

Processing images in purple folder:

Processing images in beige folder:

Processing images in yellow folder:

Processing images in white folder:

Processing images in black folder:

Processing images in tan folder:

Processing images in orange folder:

Processing images in brown folder:

Processing images in green folder:

Processing images in blue folder:

Processing images in silver folder:

Processing images in grey folder:

Processing images in pink folder:

Processing images in red folder:

Processing images in gold folder:

Processing images in purple folder:

Processing images in beige folder:

Processing images in yellow folder:

Processing images in white folder:

Processing images in black folder:

In [6]:

%%time

images = np.array(train_images)  
labels = np.array(train_labels)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

images_val=np.array(validation_images)
labels_val=np.array(validation_labels)
labels_val=label_encoder.fit_transform(labels_val)

val_inputs_t = torch.tensor(images_val, dtype=torch.float32)
val_inputs_tensor =val_inputs_t.permute(0, 3, 1, 2)
val_labels_tensor = torch.tensor(labels_val, dtype=torch.long)

dataset_val = TensorDataset(val_inputs_tensor, val_labels_tensor)

dataloader_val = DataLoader(dataset_val, batch_size=1550 ,shuffle=False)



images_t = torch.tensor(images, dtype=torch.float32)
images_tensor= images_t.permute(0, 3, 1, 2)
labels_tensor = torch.tensor(labels_encoded, dtype=torch.long)

CPU times: user 1.82 s, sys: 4.48 s, total: 6.3 s
Wall time: 13.2 s


In [17]:

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1)
        self.batch_norm1 = nn.BatchNorm2d(6) # Batch normalization after conv1
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=10, kernel_size=5, stride=1)
        self.batch_norm2 = nn.BatchNorm2d(10) 
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=10, out_channels=32, kernel_size=3, stride=1)  # New conv layer
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
       
        self.fc1 = nn.Linear(32 * 10 * 10, 128)
        self.dropout1 = nn.Dropout(0.5)  #dropout probability of 0.5
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(64, len(label_encoder.classes_)) 

    def forward(self, x):
        x = self.pool1(self.batch_norm1(nn.functional.relu(self.conv1(x))))
        x = self.pool2(self.batch_norm2(nn.functional.relu(self.conv2(x))))
        x = self.pool3(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 32 * 10 * 10)  # this flattens the tensor
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout1(x)  # Applying dropout after the first fully connected layer
        x = nn.functional.relu(self.fc2(x))
        x = self.dropout2(x)  
        x = self.fc3(x)
        return x



model = CustomCNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)


dataset = TensorDataset(images_tensor, labels_tensor)

dataloader = DataLoader(dataset, batch_size=100, shuffle=True)



In [18]:
%%time
#training
num_epochs = 15
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(dataloader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 50 == 49:  # print every 100 mini batches
            print(f"Epoch [{epoch + 1}/{num_epochs}], "
                  f"Step [{i + 1}/{len(dataloader)}], "
                  f"Loss: {running_loss / 50:.4f}")
            running_loss = 0.0

print('Finished Training')

Epoch [1/15], Step [50/73], Loss: 2.1656
Epoch [2/15], Step [50/73], Loss: 1.4506
Epoch [3/15], Step [50/73], Loss: 1.2138
Epoch [4/15], Step [50/73], Loss: 1.0870
Epoch [5/15], Step [50/73], Loss: 0.9803
Epoch [6/15], Step [50/73], Loss: 0.9425
Epoch [7/15], Step [50/73], Loss: 0.9048
Epoch [8/15], Step [50/73], Loss: 0.8764
Epoch [9/15], Step [50/73], Loss: 0.8139
Epoch [10/15], Step [50/73], Loss: 0.7688
Epoch [11/15], Step [50/73], Loss: 0.7883
Epoch [12/15], Step [50/73], Loss: 0.7289
Epoch [13/15], Step [50/73], Loss: 0.7486
Epoch [14/15], Step [50/73], Loss: 0.6999
Epoch [15/15], Step [50/73], Loss: 0.7115
Finished Training
CPU times: user 16min 27s, sys: 5min 58s, total: 22min 26s
Wall time: 3min 5s


In [12]:
correct = 0
total = 0

model.eval()

with torch.no_grad():
    for data in dataloader_val:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Validation Accuracy: {100 * accuracy:.2f}%')

Validation Accuracy: 80.40%


In [ ]:
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

all_preds = []
all_targets = []


with torch.no_grad():
    for images, targets in dataloader_val:
        images, targets = images.to(device), targets.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())


conf_matrix = confusion_matrix(all_targets, all_preds)


plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='d')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()
plt.savefig(' conf.png')

In [ ]:
#torch.save(model.state_dict(), '/Users/tolga/Desktop/CS464/Project/model_82_1.pth')


#best model optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001) batch size=64

In [ ]:
correct = 0
total = 0

model.eval()

with torch.no_grad():
    for data in dataloader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
accuracy = correct / total
print(f'Train Accuracy: {100 * accuracy:.2f}%')

In [ ]:
model.eval()

predicted_labels = []
true_labels = []

# Iterate through the validation DataLoader
with torch.no_grad():
    for data in dataloader_val:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        
        # Convert PyTorch tensors to numpy arrays
        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Generate the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

In [ ]:
ax = sns.heatmap(conf_matrix, annot=True, fmt='d', )
ax.set_xlabel("Predicted Label")
ax.set_ylabel("Actual Label")

In [ ]:
datloader_val.shape

In [ ]:
len(validation_labels)